Перед вами стоит задача – подготовить аналитический отчет для HR-отдела. На основании проведенной аналитики предполагается составить рекомендации для отдела кадров по стратегии набора персонала, а также по взаимодействию с уже имеющимися сотрудниками.
<br><br> В базе данных лежит набор таблиц, которые содержат данные о сотрудниках вымышленной компании.
Сделайте обзор штата сотрудников компании. Составьте набор предметов исследования, а затем проверьте их на данных. Вся аналитика должна быть выполена с помощью SQL. Впоследствии данные можно визуализировать, однако финальные датафреймы для графиков также должны быть подготовлены с помощью SQL. <br><br>

Примеры гипотез:
1. Есть зависимость между `perfomance score` и тем, под чьим руководством работает сотрудник.
2. Есть зависимость между продолжительностью работы в компании и семейным положением сотрудника.
2. Есть зависимость между продолжительностью работы в компании и возрастом сотрудника.

<br><br>
Параметры для подключения следующие: хост – `***, порт – `***`, имя базы данных – `human_resources`, пользователь – `***`, пароль – `***`. Таблицы, доступные для анализа, – `hr_dataset`, `production_staff`, `recruting_costs`, `salary_grid`.

In [2]:
import psycopg2
import pandas as pd

In [3]:
#Исследуем данные
pg_connection = {
    "host": "****",
    "port": ***,
    "dbname": "human_resources",
    "user": "***",
    "password": "***"
}
conn = psycopg2.connect(**pg_connection)

cursor = conn.cursor()

sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]


print("Какие таблицы содержатся в Postgres: %s" % tables_data)


table_names = [i[0] for i in tables_data][0:5]
for table in table_names:
    sql_str = "SELECT COUNT(*) FROM public.{}".format(table)
    cursor.execute(sql_str)
    row_count = [a for a in cursor.fetchall()][0][0]
    print("table {}:{}".format(table, row_count))

    
table_fields = []

sql_str = "SELECT * FROM hr_dataset LIMIT 10;"
cursor.execute(sql_str)
for i in list(cursor.description):
    new_tuple = list(i)[0]
    table_fields.append(new_tuple)

print("Поля таблицы hr_dataset: %s" % table_fields)

table_fields = []
sql_str = "SELECT * FROM production_staff LIMIT 10;"
cursor.execute(sql_str)
for i in list(cursor.description):
    new_tuple = list(i)[0]
    table_fields.append(new_tuple)

print("Поля таблицы production_staff: %s" % table_fields)

table_fields = []
sql_str = "SELECT * FROM recruiting_costs LIMIT 10;"
cursor.execute(sql_str)
for i in list(cursor.description):
    new_tuple = list(i)[0]
    table_fields.append(new_tuple)

print("Поля таблицы recruiting_costs: %s" % table_fields)


conn.commit()
cursor.close()

Какие таблицы содержатся в Postgres: [('manager_id',), ('hr_dataset_new',), ('hr_dataset',), ('production_staff_new',), ('temp_tbl1',), ('recruiting_costs_new',), ('temp_tbl2',), ('temp_tbl3',), ('my_years',), ('sum_race',), ('my_month',), ('my_dates',), ('pfm_version',), ('pfm_report',), ('pfm_form',), ('salary_grid_new',), ('production_staff',), ('amount_emps_new',), ('temp_tbl',), ('table_managers_1',), ('recruiting_costs',), ('pfm_link',), ('pfm_value_list',), ('pfm_value',), ('pfm_section',), ('pfm_attribute',), ('temp_tbl4',), ('hr_dataset_sql',), ('salary_grid',), ('source_employee',), ('table_managers',), ('use_t',), ('table_0',), ('table_1',), ('query_in',), ('query',), ('age',), ('new_age',), ('tmpDB',), ('df_count',), ('df_hr_dataset',), ('amount_emps',), ('new_age_1',), ('table_5',), ('manager',)]
table manager_id:86
table hr_dataset_new:310
table hr_dataset:310
table production_staff_new:256
table temp_tbl1:21
Поля таблицы hr_dataset: ['id', 'Employee Name', 'Employee Numb

<b>Гипотеза 1: Есть зависимость между тем, где находят сотрудника (канал) и уровнем качества работы сотрудника (оценка эффективности каналов найма)</b>

In [49]:
# рассчитываем стоимость каналов найма для компании
cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()


cursor = conn.cursor()

sql_str = 'SELECT "Employee Source",employees_hired, costs, costs/employees_hired AS per_person FROM(SELECT "Employee Source", COUNT(*)AS employees_hired FROM hr_dataset GROUP BY "Employee Source") AS headcount LEFT JOIN (SELECT "Employment Source", "Total" AS costs FROM recruiting_costs) AS hr_costs ON hr_costs."Employment Source" = headcount."Employee Source" ORDER BY per_person DESC'

cursor.execute(sql_str)
hire_cost_efficiency = pd.DataFrame(cursor.fetchall(), columns = ['Employee Sourse', 'number of employees', 'total hire costs','cost per person' ])


conn.commit()
cursor.close()
hire_cost_efficiency

,Employee Sourse,number of employees,total hire costs,cost per person
0,Indeed,8,NaN,NaN
1,Careerbuilder,1,7790.0,7790.0
2,Pay Per Click,1,1323.0,1323.0
3,MBTA ads,17,10980.0,645.0
4,On-campus Recruiting,12,7500.0,625.0
5,Website Banner Ads,13,7143.0,549.0
6,Social Networks - Facebook Twitter etc,11,5573.0,506.0
7,Newspager/Magazine,18,8291.0,460.0
8,Other,9,3995.0,443.0
9,Billboard,16,6192.0,387.0


In [50]:
# выбираем топ-5 самых дорогих для компании каналов найма, оцениваем среднее время работы в компании сотрудников, 
# нанятых через них, а также их оценки

cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()


cursor = conn.cursor()

analysis_list = tuple(hire_cost_efficiency.iloc[1:6, 0])

sql_str = 'SELECT *, SUM(employee_number) OVER(), employee_number/(SUM(employee_number) OVER())*100 FROM (SELECT "Performance Score", COUNT(*) AS employee_number, AVG("Days Employed") FROM hr_dataset WHERE "Employee Source" IN {} GROUP BY "Performance Score") AS randomtable'.format(analysis_list)

cursor.execute(sql_str)
top = pd.DataFrame(cursor.fetchall(),columns = ['Performance Score', 'employees number', 'avg days employed', 'total employees', 'percentage of employees'])


conn.commit()
cursor.close()
top.set_index(['Performance Score'])

,employees number,avg days employed,total employees,percentage of employees
Performance Score,,,,
Exceptional,2,1693.0000000000000000,44,4.54545454545454545500
90-day meets,2,1035.0000000000000000,44,4.54545454545454545500
N/A- too early to review,5,406.6000000000000000,44,11.36363636363636363600
Fully Meets,25,1627.5200000000000000,44,56.81818181818181818200
Exceeds,5,1697.0000000000000000,44,11.36363636363636363600
PIP,3,1505.3333333333333333,44,6.81818181818181818200
Needs Improvement,2,909.0000000000000000,44,4.54545454545454545500


In [51]:
# оцениваем остальные каналы, чтобы понимать, насколько сильно отличается время работы сотрудника, а также оценки
cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()


cursor = conn.cursor()



sql_str = 'SELECT *, SUM(employee_number) OVER(), employee_number/(SUM(employee_number) OVER())*100 FROM (SELECT "Performance Score", COUNT(*) AS employee_number, AVG("Days Employed") FROM hr_dataset WHERE "Employee Source" NOT IN {} GROUP BY "Performance Score") AS randomtable'.format(analysis_list)

cursor.execute(sql_str)
all_others = pd.DataFrame(cursor.fetchall(),columns = ['Performance Score', 'employees number', 'avg days employed', 'total employees', 'percentage of employees'])


conn.commit()
cursor.close()
all_others.set_index(['Performance Score'])

,employees number,avg days employed,total employees,percentage of employees
Performance Score,,,,
Exceptional,7,1703.7142857142857143,266,2.63157894736842105300
90-day meets,29,1057.2413793103448276,266,10.90225563909774436100
N/A- too early to review,32,575.1875000000000000,266,12.03007518796992481200
Fully Meets,156,1369.9871794871794872,266,58.64661654135338345900
Exceeds,23,1612.3043478260869565,266,8.64661654135338345900
PIP,6,1387.6666666666666667,266,2.25563909774436090200
Needs Improvement,13,1436.1538461538461538,266,4.88721804511278195500


<b>Рекомендации после анализа гипотезы 1:</b> Нельзя сказать, что компания эффективно использует бюджет по найму сотрудников:
        Мы проанализировали стоимость найма одного человека по разным каналам поиска и выяснили, что часть сотрудников нанимается через
        достаточно дорогие каналы, такие как Career Builder, Pay Per Click, MBTA ads. При более высоких инвестициях мы ожидаем, что нанятые 
        сотрудники будет обладать более высокой квалификацией(будут высоко оценены) или дольше будут работать внутри компании.
        Однако, проанализировав данные мы выяснили, что использование таких каналов не несет в себе преимущества для компании:
        Оценки:
        56% сотрудников, нанятых через эти каналы получили оценку Fully Meets против 58% сотрудников, нанятых через другие каналы
        4,5%сотрудников, нанятых через эти каналы получили оценку Needs Improvement против 4,8% сотрудников, нанятых через другие каналы
        При этом среднее время работы в компании у двух групп сотрудников также значительно не отличается.
        Мы рекомендуем компании по возможности не использовать Career Builder, Pay Per Click, MBTA ads, On-campus Recruiting, Website Banner Ads
        в связи с высокой стоимостью и неочевидной выгодой для компании


<b>Гипотеза 2: В компании сущесвует высокая скорость смены кадров: большая доля сотрудников увольняетсяв короткие сроки после найма. Из-за этого компания испытывает кадровый дефицит и вынуждена использовать дополнительный бюджет для поиска новых сотрудников </b>

In [52]:
# расчет количества уволившихся

cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()

cursor = conn.cursor()

sql_str = 'SELECT termination_year, COUNT(*) FROM(SELECT *, CAST(EXTRACT(YEAR FROM CAST("Date of Termination" AS DATE)) AS INTEGER) AS termination_year FROM hr_dataset WHERE "Date of Termination" IS NOT NULL) AS tablename GROUP BY termination_year ORDER BY termination_year'
cursor.execute(sql_str)
termination_dynamics = pd.DataFrame(cursor.fetchall(), columns = ['year','number of employees fired'])


conn.commit()
cursor.close()

termination_dynamics

,year,number of employees fired
0,2010,2
1,2011,14
2,2012,17
3,2013,15
4,2014,14
5,2015,27
6,2016,14


In [53]:
# расчет количества нанятых

cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()

cursor = conn.cursor()

sql_str = 'SELECT hire_year, COUNT(*) FROM(SELECT *, CAST(EXTRACT(YEAR FROM CAST("Date of Hire" AS DATE)) AS INTEGER) AS hire_year FROM hr_dataset) AS tablename GROUP BY hire_year ORDER BY hire_year'
cursor.execute(sql_str)
hire_dynamics = pd.DataFrame(cursor.fetchall(), columns = ['year','number of employees hired'])


conn.commit()
cursor.close()

hire_dynamics

,year,number of employees hired
0,2006,1
1,2007,2
2,2008,3
3,2009,7
4,2010,9
5,2011,84
6,2012,44
7,2013,44
8,2014,60
9,2015,36


In [54]:
# количество нанятых и уволившихся в том же году в зависимости от менеджера

cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()

cursor = conn.cursor()

sql_str = 'SELECT "Manager Name", "Reason For Term", hire_year, COUNT(*) FROM(SELECT *, CAST(EXTRACT(YEAR FROM CAST("Date of Hire" AS DATE)) AS INTEGER) AS hire_year, CAST(EXTRACT(YEAR FROM CAST("Date of Termination" AS DATE)) AS INTEGER) AS termination_year FROM hr_dataset) AS tablename WHERE hire_year = termination_year GROUP BY hire_year, termination_year,"Manager Name","Reason For Term" ORDER BY hire_year'
cursor.execute(sql_str)
terminated_same_year = pd.DataFrame(cursor.fetchall(), columns = ['Manager',"Reason For Term",'year','number of employees'])


conn.commit()
cursor.close()

terminated_same_year

,Manager,Reason For Term,year,number of employees
0,Amy Dunn,Another position,2011,1
1,Amy Dunn,return to school,2011,1
2,Amy Dunn,unhappy,2011,1
3,David Stanley,Another position,2011,1
4,David Stanley,return to school,2011,1
5,Elijiah Gray,hours,2011,1
6,Kelley Spirea,attendance,2011,1
7,Ketsia Liebig,Another position,2011,1
8,Kissy Sullivan,Another position,2011,1
9,Michael Albert,unhappy,2011,1


<b>Рекомендации после анализа гипотезы 2</b> 
<br>
Гипотеза 2 не подтвердилась в процессе анализа - компания росла и активно нанимала сотрудников до 2016 года,
Количество уволенных сотрудников также росло до 2016 года. В 2016 году коичество уволившихся сотрудников уменьшилось, также как уменьшилось количество нанятых сотрудников.
За всю историю анализа компания не испытывала кадрового дефицита.
Мы также рассмотрели количество сотрудников, которые уволились в тот же год, в котором были наняты: 
количество таких сотрудников росло до 2016 года.  В 2016-2017 году ни один из нанятых сотрудников не уовлился в течение года
<br>
Мы проанализировали, зависело ли количество уволенных в тот же год сотрудников от конкретного менеджера:
Мы выявили, что в 2015 году 4 сотрудника под руководством Simon Roup уволились в тот же год (из 6 за весь год).
Однако, при проверке причин увольнения,  мы выявили, что 2 сотрудника из команды этого менеджера были уволены по причине неявки (no-call, no-show).

Таким образом, мы не можем сделать вывод о том, что компания испытывает кажровый дефицит, существует необходимость постоянного поиска сотрудников, а также что текучка кадров вызвана конкретным менеджером


<b>Гипотеза 3:

Заработная плата сотрудников зависит от опыта работы, оценки и качества работы на производстве. Оценка производственного персонала при этом отражает качество работы сотрудника на производстве 

</b>

In [95]:
# рассчитываем среднюю зарплату по позициям
cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()

cursor = conn.cursor()

sql_str = 'SELECT "Position", COUNT(*), AVG(randomtable.Pay_rate) FROM (SELECT *, CAST("Pay" AS NUMERIC) AS Pay_rate FROM production_staff WHERE "TermDate" IS NULL AND "Employee Name" IS NOT NULL) AS randomtable GROUP BY "Position"'
cursor.execute(sql_str)
by_postion = pd.DataFrame(cursor.fetchall(), columns = ['Position', 'Number of Employees', 'Average pay rate'])


conn.commit()
cursor.close()

by_postion


,Position,Number of Employees,Average pay rate
0,Production Manager,9,53.6111111111111111
1,Production Technician II,32,25.1715625000000000
2,Production Technician I,85,19.2495294117647059


In [96]:
# рассчитываем среднюю зарплату по оценкам
cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()

cursor = conn.cursor()

sql_str = 'SELECT "Position", "Performance Score" ,COUNT(*), AVG(randomtable.Pay_rate) FROM (SELECT *, CAST("Pay" AS NUMERIC) AS Pay_rate FROM production_staff WHERE "TermDate" IS NULL AND "Employee Name" IS NOT NULL) AS randomtable GROUP BY "Position", "Performance Score" ORDER BY "Position"'
cursor.execute(sql_str)
dataset = pd.DataFrame(cursor.fetchall(), columns = ['position', 'Performance', 'number of employees', 'Pay rate'])


conn.commit()
cursor.close()

dataset


,position,Performance,number of employees,Pay rate
0,Production Manager,Needs Improvement,1,53.0000000000000000
1,Production Manager,Fully Meets,6,53.2500000000000000
2,Production Manager,Exceeds,2,55.0000000000000000
3,Production Technician I,90-day meets,6,19.1250000000000000
4,Production Technician I,Exceeds,9,18.4166666666666667
5,Production Technician I,Needs Improvement,2,22.0000000000000000
6,Production Technician I,Fully Meets,55,19.1992727272727273
7,Production Technician I,Exceptional,2,20.0000000000000000
8,Production Technician I,PIP,2,18.0000000000000000
9,Production Technician I,N/A- too early to review,9,19.9722222222222222


In [97]:
# рассчитываем среднюю зарплату по количеству лет опыта
cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()

cursor = conn.cursor()

sql_str = 'SELECT "Position", "year_work", COUNT(*), AVG(randomtable.Pay_rate) FROM (SELECT *, CAST("Pay" AS NUMERIC) AS Pay_rate, CAST((make_date(2017, 01, 01)-CAST("Date of Hire" AS DATE))/365 AS INTEGER) AS year_work FROM production_staff WHERE "TermDate" IS NULL AND "Employee Name" IS NOT NULL) AS randomtable GROUP BY "Position", "year_work" ORDER BY "Position", "year_work"'
cursor.execute(sql_str)
dataset = pd.DataFrame(cursor.fetchall(), columns = ['position', 'number of years employed', 'number of employees', 'average pay rate'])


conn.commit()
cursor.close()

dataset


,position,number of years employed,number of employees,average pay rate
0,Production Manager,0,1,55.0000000000000000
1,Production Manager,1,1,54.0000000000000000
2,Production Manager,2,1,51.0000000000000000
3,Production Manager,3,1,55.0000000000000000
4,Production Manager,4,2,52.5000000000000000
5,Production Manager,5,1,54.5000000000000000
6,Production Manager,6,1,53.0000000000000000
7,Production Manager,7,1,55.0000000000000000
8,Production Technician I,0,3,21.5000000000000000
9,Production Technician I,1,8,18.8437500000000000


In [98]:
# рассчитывем среднее, максимальное, минимальное количество ошибок по оценкам 
cursor = conn.cursor()
cursor.execute("ROLLBACK")
conn.commit()

cursor = conn.cursor()

sql_str = 'SELECT "Performance Score", MIN("Daily Error Rate"), MAX("Daily Error Rate"), AVG("Daily Error Rate"), COUNT(*) FROM production_staff WHERE "TermDate" IS NULL AND "Employee Name" IS NOT NULL GROUP BY "Performance Score"'
cursor.execute(sql_str)
production_staff_dataset = pd.DataFrame(cursor.fetchall(), columns = ['Performance', 'Min number of errors', 'Max number of errors', 'average number of errors', 'number of employees'])


conn.commit()
cursor.close()

production_staff_dataset

,Performance,Min number of errors,Max number of errors,average number of errors,number of employees
0,Exceptional,0,1,0.50000000000000000000,4
1,90-day meets,0,4,1.2857142857142857,7
2,N/A- too early to review,0,3,1.3076923076923077,13
3,Fully Meets,0,3,0.68354430379746835443,80
4,Exceeds,0,4,0.40000000000000000000,15
5,PIP,0,3,1.3333333333333333,3
6,Needs Improvement,0,4,1.7500000000000000,4


<b>Рекомендации после анализа гипотезы 3:</b>
<br>
Мы проанализировали зависимость между оценкой и средней заработной патой на различных позициях, однако мы не виявили однозначносй зависимости между этими показателями.
Сотрудники с оценкой Needs Improvement, хотя их не так много, зачастую получают зарплату большую, чем сотрудники с оценками Exceptional/Exceed/Fully meets
При этом мы также проанаизировали - отражает ли оценка, выставенная руководителем уровень ошибок, которые совершает сотрудник ежедневно.
На уровне среднего уровня ошибок сотрудников видно, что в целом оценка отражает реальные показатели.
Однако есть сотрудники с оценками Exceptional и Exceeds, где уровень ошибок высок для такой оценки - от 1 до 4.
Такая же ситуация с низкими оценками: сотрудники с Needs Improvement, где уровень ошибок составляет 0  

Мы также проанализировали зависимость между сроком работы в компании и заработной платы. За текущую дату мы принимаем 01.01.2017, т.к. последние активности (увольнения и принятия на работу) сдержатся за 2016 год
Мы не пришли к очевидному результату: сотрудники, приходящие на должность могут получать зарпату выше, чем сотрудники, пришедшие 6 лет назад.

Мы рекомендовали бы автоматизированную систему оценки сотрудников, работающих на производстве.
В отличие от работы офисных сотрудников, работа производственных функций может быть оценена на основе объективных данных, например, посещаемость и ошибки.
Такая система, помимо экономии ресурсов менеджеров на выставление оценки, более понятна самим сотрудникам и не подвержена субъективным искажениям руководителей.
Дополнитеьно, мы можем порекомендовать ревизию по заработной плате работников: при одинаковых должностях, боьшем опыте работы и высокой оценке, сотрудник может получать меньшую плату, чем недавно нанятый сотрудник